In [1]:
pip install PyObjC


  Obtaining dependency information for PyObjC from https://files.pythonhosted.org/packages/cb/a5/4b9ed66894d804dc57c13b7fdb8f65ed831f13514216ce736e1455dfe214/pyobjc-10.3.1-py3-none-any.whl.metadata
     ---------------------------------------- 0.0/935.0 kB ? eta -:--:--
     ---------- --------------------------- 256.0/935.0 kB 5.2 MB/s eta 0:00:01
     ------------------------------------- 935.0/935.0 kB 11.9 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'error'


  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [2 lines of output]
      running egg_info
      error: PyObjC requires macOS to build
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: subprocess-exited-with-error

× Getting requirements to build wheel did not run successfully.
│ exit code: 1
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.


In [2]:
# import necessary libraries
import speech_recognition as sr
from gtts import gTTS
import playsound
import nltk
from nltk.tokenize import word_tokenize
import csv
import os
import gradio as gr

# Download necessary NLTK data
nltk.download('punkt')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\lovel\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
# Load inventory from CSV
def load_inventory(file_path):
    # Initialize an empty dictionary to hold data
    inventory = {}
    # Open CSV file in read mode (r)
    with open(file_path, mode='r') as file:
        reader = csv.DictReader(file)
        # Iterate of each row in the CSV file
        for row in reader:
            # Extract the product name from the current row
            product = row["product"]
            # Add the product to the inventory dictionary with its qty and price
            inventory[product] = {"quantity": int(row["quantity"]), "price": float(row["price"])}
            # Return populated inventory dictionary
    return inventory

# Update the file path to the resources folder
csv_file_path = os.path.join('Resources', 'mock_inventory.csv')
inventory = load_inventory(csv_file_path)


In [4]:
# create a function that will recognize when person is speaking
def listen(audio_file_path):
    print(f"Received audio file path: {audio_file_path}")
    # Creating a Recognizer instance to recognize speech
    recognizer = sr.Recognizer()
    try:
    # Use computer microphone as the source of the audio
        with sr.AudioFile(audio_file_path) as source:
        # this lets the user know that the assistant is listening
        # print("Listening...")
        # captures the audio that comes in from mic
            audio = recognizer.record(source)
            # Uses Google's speech recognition to transcribe the audio to text
            text = recognizer.recognize_google(audio)
            print(f"Recognized text: {text}")
            # prints the transcribed text
            # print(f"User said: {text}")
            # return the transcribed text
            return text
    except sr.UnknownValueError:
            print("Error: Could not understand the audio")
            # how to handle where speech was not understood
            # print("Sorry, I did not understand that.")
            # empty string shows a failure to understand
            return "Sorry, I did not understand that."
    except sr.RequestError:
            print("Error: Could not request results from Google Speech Recognition service.")
            return "Sorry, the service is down."
    except AssertionError as e:
         print(f"AssertionError:{e}")
         return "Sorry the audio file is not valid."

def speak(text):
    if not text:
         text = "Sorry, I have nothing to say."
    # Creates a gTTS (Google Text-to-Speech) instance with provided text
    tts = gTTS(text=text, lang='en')
    # Define the filename for the audio file
    filename = "response.mp3"
    # Save to mp3 file
    tts.save(filename)
    # play the mp3 file to speak the captured text
    try:
        playsound.playsound(filename)
        print(f"Playing sound: {filename}")
    except Exception as e:
        print(f"Error playing sound: {e}")
    return filename


In [5]:
# create a function that will understand commands and reply with appropriate response
# this will need to be updated as of 7/21, some responses are not relevant.
def process_command(command):
    print(f"Processing command: {command}")
    tokens = word_tokenize(command.lower())
    if "find" in tokens and "product" in tokens:
        response = "Sure, I can help you find a product. What are you looking for?"
    elif "check" in tokens and "stock" in tokens:
        product_name = extract_product_name(tokens)
        if product_name in inventory:
            quantity = inventory[product_name]["quantity"]
            response = f"We have {quantity} {product_name}(s) in stock."
        else:
            response = f"Sorry, we don't have {product_name} in our inventory."
    elif "information" in tokens and "paint" in tokens:
        response = "I can provide information about our paints and finishes. What would you like to know?"
    else:
        response = "I'm sorry, I didn't understand that. Can you please repeat?"
    print(f"Response: {response}")
    return response



In [6]:
def extract_product_name(tokens):
    # Iterate over each token in the command
    for token in tokens:
        # Check if the token matches any product name in inventory
        for product in inventory.keys():
            if token in product.lower():
                # Return the product name if a match is found
                return product
            # Return an empty string if no match is found
    return ""

In [7]:
def assistant(audio_file_path):
    print(f"Assistant received audio file path: {audio_file_path}")
    command = listen(audio_file_path)
    response = process_command(command)
    audio_response = speak(response)
    return response, audio_response

In [ ]:
# Create Gradio interface
iface = gr.Interface(
    fn=assistant, 
    inputs=gr.Audio("microphone", type="filepath"),
    outputs=[gr.Textbox(), gr.Audio(type="filepath")]
)

iface.launch(share=True)

Running on local URL:  http://127.0.0.1:7861
Running on public URL: https://944d86e546f5850f4d.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Assistant received audio file path: C:\Users\lovel\AppData\Local\Temp\gradio\bf6cf805903fa56e83a7e2f88b20ffe87d7fa6e4\audio.wav
Received audio file path: C:\Users\lovel\AppData\Local\Temp\gradio\bf6cf805903fa56e83a7e2f88b20ffe87d7fa6e4\audio.wav
Recognized text: how much is Regal interior Flats
Processing command: how much is Regal interior Flats
Response: I'm sorry, I didn't understand that. Can you please repeat?
Playing sound: response.mp3
Assistant received audio file path: C:\Users\lovel\AppData\Local\Temp\gradio\cefed23980d6ef7ea28cea63562bea113aeacd5b\audio.wav
Received audio file path: C:\Users\lovel\AppData\Local\Temp\gradio\cefed23980d6ef7ea28cea63562bea113aeacd5b\audio.wav
Recognized text: how much is the Regal interior mapping
Processing command: how much is the Regal interior mapping
Response: I'm sorry, I didn't understand that. Can you please repeat?



    Error 263 for command:
        open response.mp3
    The specified device is not open or is not recognized by MCI.

    Error 263 for command:
        close response.mp3
    The specified device is not open or is not recognized by MCI.
Failed to close the file: response.mp3


Error playing sound: 
    Error 263 for command:
        open response.mp3
    The specified device is not open or is not recognized by MCI.
Assistant received audio file path: C:\Users\lovel\AppData\Local\Temp\gradio\9c8afdd189b89aa07a248156ac8dc353ed1dfd1c\audio.wav
Received audio file path: C:\Users\lovel\AppData\Local\Temp\gradio\9c8afdd189b89aa07a248156ac8dc353ed1dfd1c\audio.wav
Recognized text: do you have Regal interior flat
Processing command: do you have Regal interior flat
Response: I'm sorry, I didn't understand that. Can you please repeat?



    Error 263 for command:
        open response.mp3
    The specified device is not open or is not recognized by MCI.

    Error 263 for command:
        close response.mp3
    The specified device is not open or is not recognized by MCI.
Failed to close the file: response.mp3


Error playing sound: 
    Error 263 for command:
        open response.mp3
    The specified device is not open or is not recognized by MCI.


In [8]:
# This block of code ensures that the following statements are only executed 
# when this script is run directly, and not when it is imported as a module in another script.
#if __name__ == "__main__":
    # This creates an infinite loop to continuously listen for and process commands.
    #while True:
        # Call the listen() function to capture and transcribe spoken commands from the user.
        # The transcribed text is stored in the variable 'command'.
        #command = listen()
        # Check if a command was successfully transcribed (i.e., if 'command' is not empty).
        #if command:
            # If a valid command was captured, call the process_command() function to handle
            # the command and provide the appropriate response
            #process_command(command)
